##### https://www.kaggle.com/code/imegirin/multivariate-time-series-modeling-with-pytorch
##### https://github.com/jeffheaton/app_deep_learning/blob/main/t81_558_class_10_2_lstm.ipynb
##### https://github.com/curiousily/Getting-Things-Done-with-Pytorch
##### https://www.kaggle.com/code/megner/pytorch-lightning-lstm

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim



In [2]:
# load the dataset
dataframe = pd.read_csv('archive/AirPassengers.csv', parse_dates=['Month'])
dataframe = dataframe.sort_values(by='Month').reset_index(drop=True)
dataframe['year'] = dataframe['Month'].dt.year
dataframe['month'] = dataframe['Month'].dt.month
dataframe.head()

,Month,#Passengers,year,month
0,1949-01-01,112,1949,1
1,1949-02-01,118,1949,2
2,1949-03-01,132,1949,3
3,1949-04-01,129,1949,4
4,1949-05-01,121,1949,5


In [3]:
# nomalization
df_values = dataframe['#Passengers']
min_value = df_values.min()
max_value = df_values.max()

df_norm = (df_values - min_value) / (max_value - min_value)
df_norm.head()
df_final = pd.concat([df_norm, dataframe[['year', 'month']]], axis=1)
df_final.head()

,#Passengers,year,month
0,0.015444,1949,1
1,0.027027,1949,2
2,0.054054,1949,3
3,0.048263,1949,4
4,0.032819,1949,5


In [10]:
class MultiVariableDataset(Dataset):
    def __init__(self, df_raw, features, target, seq_len):
        self.data = df_raw
        self.seq_len = seq_len
        self.y = torch.tensor(self.data[target].values).float()
        self.X = torch.tensor(self.data[features].values).float()

    def __len__(self):
        return self.data.shape[0] - self.seq_len -1

    def __getitem__(self, idx):
        features = self.X[idx:(idx + self.seq_len)]
        target = self.y[idx + self.seq_len]  # the return value is numpy float
        target = target.unsqueeze(0)
       
        return features, target
        
train_size = int(len(df_final) * 0.66)
valid_size = len(df_final) - train_size

features = ['year', 'month', '#Passengers']
target = '#Passengers'

# print(df_final.loc[:train_size,:])
train_dataset = MultiVariableDataset(df_final.loc[0:train_size,:], features, target, 4)
                             
valid_dataset = MultiVariableDataset(df_final.loc[train_size:len(df_final),:], features, target, 4)


In [11]:
# Create data loader
dataloader = DataLoader(train_dataset, batch_size=4, shuffle=False)
# Get one batch of data
dataiter = iter(dataloader)
inputs, target = next(dataiter)
print(inputs[:,:,1])
print(target[:,0])


tensor([[1., 2., 3., 4.],
        [2., 3., 4., 5.],
        [3., 4., 5., 6.],
        [4., 5., 6., 7.]])
tensor([0.0328, 0.0598, 0.0849, 0.0849])


In [1]:
# import pandas as pd
# from sklearn.preprocessing import MinMaxScaler


# scaler = MinMaxScaler()

# dfTest = pd.DataFrame({'A':[14.00,90.20,90.95,96.27,91.21],
#                            'B':[103.02,107.26,110.35,114.23,114.68],
#                            'C':['big','small','big','small','small']})

# dfTest[['A', 'B']] = scaler.fit_transform(dfTest[['A', 'B']])

# dfTest = pd.DataFrame({'A':[14.00,90.20,90.95,96.27,91.21],'B':[103.02,107.26,110.35,114.23,114.68], 'C':['big','small','big','small','small']})
# df = dfTest[['A', 'B']]
# df_norm = (df - df.min()) / (df.max() - df.min())
# print df_norm
# print pd.concat((df_norm, dfTest.C),1)